python run.py -c 10 -i /home/ubuntu/OCR/corpus/rus_corpus.txt -ft /home/ubuntu/OCR/fonts/ocr_russia.ttf -f 48 -wd 320


In [10]:
# !cd /home/ubuntu/OCR/TextRecognitionDataGenerator/trdg
# !python /home/ubuntu/OCR/TextRecognitionDataGenerator/trdg/run.py -c 10 -i /home/ubuntu/OCR/corpus/rus_corpus.txt -ft /home/ubuntu/OCR/fonts/ocr_russia.ttf -f 48 -wd 320

Missing modules for handwritten text generation.
100%|██████████████████████████████████████████| 10/10 [00:00<00:00, 192.54it/s]


In [31]:
import sys
import os
import re
import random
# Добавляем директорию, содержащую модуль trdg
sys.path.append('/home/ubuntu/OCR/TextRecognitionDataGenerator')
# Теперь можно импортировать
from trdg.generators import GeneratorFromStrings

# Путь к файлу с текстом и шрифтам
text_file = "/home/ubuntu/OCR/corpus/rus_corpus.txt"
font_path1 = "/home/ubuntu/OCR/fonts/ocr_russia.ttf"
font_path2 = "/home/ubuntu/OCR/fonts/ocr_calibri.ttf"

# Чтение текста из файла
with open(text_file, "r", encoding="utf-8") as f:
    strings = [line.strip() for line in f if line.strip()]

# Ограничим количество слов, если корпус слишком большой
max_words = 100  # Вы можете изменить это значение
if len(strings) > max_words:
    # Случайно выбираем подмножество слов
    strings = random.sample(strings, max_words)

# Установка выходной директории
output_dir = "/home/ubuntu/OCR/TextRecognitionDataGenerator/trdg/out"
os.makedirs(output_dir, exist_ok=True)

# Создаем файл для аннотаций Paddle OCR
annotation_file = os.path.join(output_dir, "paddle_ocr_labels.txt")

# Набор аугментаций для разнообразия
augmentations = [
    {"name": "normal", "random_skew": False, "blur": 0, "distorsion_type": 0},
    {"name": "random_skew", "random_skew": True, "blur": 0, "distorsion_type": 0},
    {"name": "blur_light", "random_skew": False, "blur": 1, "distorsion_type": 0},
    {"name": "blur_medium", "random_skew": False, "blur": 2, "distorsion_type": 0},
    {"name": "blur_heavy", "random_skew": False, "blur": 3, "distorsion_type": 0},
    {"name": "distortion_sin", "random_skew": False, "blur": 0, "distorsion_type": 1},
    {"name": "distortion_cos", "random_skew": False, "blur": 0, "distorsion_type": 2},
    {"name": "distortion_random", "random_skew": False, "blur": 0, "distorsion_type": 3},
    {"name": "combined1", "random_skew": True, "blur": 1, "distorsion_type": 1},
    {"name": "combined2", "random_skew": True, "blur": 2, "distorsion_type": 2}
]

# Создаем счетчик для общего количества изображений
total_images = 0

# Открываем файл для записи аннотаций
with open(annotation_file, "w", encoding="utf-8") as anno_file:
    # Обрабатываем каждое слово отдельно
    for word_idx, word in enumerate(strings):
        # Для каждого шрифта
        for font_idx, font_path in enumerate([font_path1, font_path2]):
            font_name = "russia" if font_idx == 0 else "calibri"
            
            # Для каждой аугментации
            for aug_idx, aug in enumerate(augmentations):
                # Создаем генератор только для текущего слова
                generator = GeneratorFromStrings(
                    [word],  # Передаем только одно слово
                    count=1,  # Нам нужен только один экземпляр
                    fonts=[font_path],
                    size=48,
                    width=320,
                    language="ru",
                    random_skew=aug["random_skew"],
                    blur=aug["blur"],
                    distorsion_type=aug["distorsion_type"],
                    distorsion_orientation=1 if aug["distorsion_type"] > 0 else 0
                )
                
                # Генерируем изображение
                for img, text in generator:
                    # Очищаем текст от специальных символов для имени файла
                    safe_text = re.sub(r'[^\w\s-]', '', text)
                    safe_text = re.sub(r'\s+', '_', safe_text)
                    
                    if len(safe_text) > 30:
                        safe_text = safe_text[:30]
                    
                    if not safe_text:
                        safe_text = f"image_{total_images}"
                    
                    # Создаем имя файла с указанием слова, шрифта и типа аугментации
                    file_name = f"{safe_text}_{font_name}_{aug['name']}_{word_idx}"
                    image_path = f"{output_dir}/{file_name}.jpg"
                    
                    # Сохраняем изображение
                    img.save(image_path)
                    
                    # Записываем аннотацию
                    rel_image_path = os.path.relpath(image_path, start=os.path.dirname(output_dir))
                    anno_file.write(f"{rel_image_path}\t{text}\n")
                    
                    total_images += 1
                    
                    # Показываем прогресс каждые 100 изображений
                    if total_images % 100 == 0:
                        print(f"Сгенерировано {total_images} изображений...")

print(f"\nВсего сгенерировано {total_images} изображений:")
print(f"- Обработано {len(strings)} слов из корпуса")
print(f"- Для каждого слова создано {len(augmentations) * 2} вариантов (2 шрифта × 10 аугментаций)")
print(f"- Использованы шрифты: ocr_russia.ttf и ocr_calibri.ttf")
print(f"- Аугментации включают: нормальные изображения, случайные наклоны, размытие и искажения")
print(f"Все изображения сохранены в папке: {output_dir}")
print(f"Создан файл аннотаций для Paddle OCR: {annotation_file}")

Сгенерировано 100 изображений...
Сгенерировано 200 изображений...
Сгенерировано 300 изображений...
Сгенерировано 400 изображений...
Сгенерировано 500 изображений...
Сгенерировано 600 изображений...
Сгенерировано 700 изображений...
Сгенерировано 800 изображений...
Сгенерировано 900 изображений...
Сгенерировано 1000 изображений...
Сгенерировано 1100 изображений...
Сгенерировано 1200 изображений...
Сгенерировано 1300 изображений...
Сгенерировано 1400 изображений...
Сгенерировано 1500 изображений...
Сгенерировано 1600 изображений...
Сгенерировано 1700 изображений...
Сгенерировано 1800 изображений...
Сгенерировано 1900 изображений...
Сгенерировано 2000 изображений...

Всего сгенерировано 2000 изображений:
- Обработано 100 слов из корпуса
- Для каждого слова создано 20 вариантов (2 шрифта × 10 аугментаций)
- Использованы шрифты: ocr_russia.ttf и ocr_calibri.ttf
- Аугментации включают: нормальные изображения, случайные наклоны, размытие и искажения
Все изображения сохранены в папке: /home/ubu

In [32]:
# Путь к файлу аннотаций
file_path = "/home/ubuntu/OCR/data/rec/rec_gt_train1.txt"

# Префиксы для замены
old_prefix = "out/"
new_prefix = "/home/ubuntu/OCR/data/rec/train1/"

# Чтение содержимого файла
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

# Замена префикса
modified_content = content.replace(old_prefix, new_prefix)

# Запись измененного содержимого обратно в файл
with open(file_path, "w", encoding="utf-8") as f:
    f.write(modified_content)

print(f"Префикс '{old_prefix}' заменен на '{new_prefix}' в файле {file_path}")

Префикс 'out/' заменен на '/home/ubuntu/OCR/data/rec/train1/' в файле /home/ubuntu/OCR/data/rec/rec_gt_train1.txt


In [33]:
import os
import shutil

# Пути к директориям и файлам
train_dir = "/home/ubuntu/OCR/data/rec/train1"
test_dir = "/home/ubuntu/OCR/data/rec/test1"
train_anno_path = "/home/ubuntu/OCR/data/rec/rec_gt_train1.txt"
test_anno_path = "/home/ubuntu/OCR/data/rec/rec_gt_test1.txt"

# Создаем директорию test1, если её нет
os.makedirs(test_dir, exist_ok=True)

# Чтение первых 10 строк из файла аннотаций
with open(train_anno_path, "r", encoding="utf-8") as f:
    annotations = [line.strip() for line in f.readlines()[:10]]

# Подготовка новых аннотаций для test1
test_annotations = []

# Копирование файлов и создание новых аннотаций
for annotation in annotations:
    parts = annotation.split("\t")
    if len(parts) == 2:
        train_path, text = parts
        
        # Получаем имя файла
        file_name = os.path.basename(train_path)
        
        # Создаем путь к файлу в test1
        test_path = os.path.join(test_dir, file_name)
        
        # Копируем файл
        if os.path.exists(train_path):
            shutil.copy2(train_path, test_path)
            print(f"Скопирован файл: {file_name}")
            
            # Создаем новую аннотацию с путем к test1
            test_anno_line = f"/home/ubuntu/OCR/data/rec/test1/{file_name}\t{text}"
            test_annotations.append(test_anno_line)
        else:
            print(f"Файл не найден: {train_path}")

# Записываем новые аннотации в файл rec_gt_test1.txt
with open(test_anno_path, "w", encoding="utf-8") as f:
    for anno in test_annotations:
        f.write(f"{anno}\n")

print(f"\nСкопировано {len(test_annotations)} файлов из {train_dir} в {test_dir}")
print(f"Создан файл аннотаций: {test_anno_path}")

Скопирован файл: Гемоглобин_russia_normal_0.jpg
Скопирован файл: Гемоглобин_russia_random_skew_0.jpg
Скопирован файл: Гемоглобин_russia_blur_light_0.jpg
Скопирован файл: Гемоглобин_russia_blur_medium_0.jpg
Скопирован файл: Гемоглобин_russia_blur_heavy_0.jpg
Скопирован файл: Гемоглобин_russia_distortion_sin_0.jpg
Скопирован файл: Гемоглобин_russia_distortion_cos_0.jpg
Скопирован файл: Гемоглобин_russia_distortion_random_0.jpg
Скопирован файл: Гемоглобин_russia_combined1_0.jpg
Скопирован файл: Гемоглобин_russia_combined2_0.jpg

Скопировано 10 файлов из /home/ubuntu/OCR/data/rec/train1 в /home/ubuntu/OCR/data/rec/test1
Создан файл аннотаций: /home/ubuntu/OCR/data/rec/rec_gt_test1.txt
